In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
BATCH_SIZE = 200
IMG_SIZE = (28, 28)
IMG_SHAPE = IMG_SIZE + (1,)
EPOCHS = 30
NUM_CLASS = 10

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
data_augm = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.04, 
                                                     interpolation='bilinear', 
                                                     seed=42), # 15°  2pi * 0.04 -> rad
    layers.experimental.preprocessing.RandomTranslation(height_factor=0.1,  # %
                                                        width_factor=0.2,
                                                        interpolation='nearest'),
    layers.experimental.preprocessing.RandomZoom(height_factor=0.1,
                                                 width_factor=0.1,
                                                 interpolation='nearest')
])

In [7]:
inputs = tf.keras.Input(shape=IMG_SHAPE) 
x = data_augm(inputs)
x = tf.keras.layers.Conv2D(32, (3,3))(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(32, (3,3))(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(64, (3,3))(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64, (3,3))(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(NUM_CLASS)(x)
x = tf.keras.layers.Activation('softmax')(x)
model = tf.keras.Model(inputs, x)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0     

In [9]:
model.compile(optimizer='adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [10]:
model.loss.get_config()

{'reduction': 'auto',
 'name': 'sparse_categorical_crossentropy',
 'from_logits': False}

In [11]:
model.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.001,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [12]:
history = model.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)

Epoch 1/30
300/300 [==============================] - 31s 101ms/step - loss: 0.7619 - sparse_categorical_accuracy: 0.8235
Epoch 2/30
300/300 [==============================] - 30s 101ms/step - loss: 0.1329 - sparse_categorical_accuracy: 0.9594
Epoch 3/30
300/300 [==============================] - 31s 103ms/step - loss: 0.1063 - sparse_categorical_accuracy: 0.9675
Epoch 4/30
300/300 [==============================] - 32s 107ms/step - loss: 0.0863 - sparse_categorical_accuracy: 0.9739
Epoch 5/30
300/300 [==============================] - 38s 126ms/step - loss: 0.0741 - sparse_categorical_accuracy: 0.9774
Epoch 6/30
300/300 [==============================] - 35s 117ms/step - loss: 0.0707 - sparse_categorical_accuracy: 0.9783
Epoch 7/30
300/300 [==============================] - 37s 124ms/step - loss: 0.0642 - sparse_categorical_accuracy: 0.9803
Epoch 8/30
300/300 [==============================] - 39s 129ms/step - loss: 0.0573 - sparse_categorical_accuracy: 0.9824
Epoch 9/30
300/300 [====

In [16]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0209 - sparse_categorical_accuracy: 0.9938


[0.020894015207886696, 0.9937999844551086]

In [15]:
model.save('exp_short.h5')